In [ ]:
import numpy as np
import random

NUM_CITIES = 5  # Number of cities for the TSP
NUM_ANTS = 10   # Number of ants
NUM_ITERATIONS = 100
EVAPORATION_RATE = 0.5
ALPHA = 1.0     # Influence of pheromone
BETA = 2.0      # Influence of distance
Q = 100         # Constant related to pheromone deposit

dist_matrix = np.random.randint(1, 100, size=(NUM_CITIES, NUM_CITIES))

np.fill_diagonal(dist_matrix, 0)

pheromone_matrix = np.ones((NUM_CITIES, NUM_CITIES))

def ant_colony_optimization():
    global pheromone_matrix  # Explicitly declare global usage
    best_route = None
    best_length = float('inf')

    for iteration in range(NUM_ITERATIONS):
        all_routes = []
        all_lengths = []

        for ant in range(NUM_ANTS):
            route = [random.randint(0, NUM_CITIES - 1)]
            while len(route) < NUM_CITIES:
                current_city = route[-1]
                probabilities = []
                for next_city in range(NUM_CITIES):
                    if next_city not in route:
                        pheromone = pheromone_matrix[current_city][next_city] ** ALPHA
                        distance = dist_matrix[current_city][next_city]
                        if distance > 0:
                            distance = distance ** (-BETA)
                        else:
                            distance = 0
                        probabilities.append(pheromone * distance)
                    else:
                        probabilities.append(0)

                total_prob = sum(probabilities)
                if total_prob == 0:
                    break  # No valid move
                probabilities = [p / total_prob for p in probabilities]

                next_city = np.random.choice(range(NUM_CITIES), p=probabilities)
                route.append(next_city)

            # Calculate the length of the route
            route_length = sum([dist_matrix[route[i]][route[i + 1]] for i in range(NUM_CITIES - 1)]) + \
                           dist_matrix[route[-1]][route[0]]  # Closing the loop back to start

            all_routes.append(route)
            all_lengths.append(route_length)

            if route_length < best_length:
                best_length = route_length
                best_route = route

        # Pheromone update
        pheromone_matrix *= (1 - EVAPORATION_RATE)
        for route, route_length in zip(all_routes, all_lengths):
            for i in range(NUM_CITIES - 1):
                pheromone_matrix[route[i]][route[i + 1]] += Q / route_length
            pheromone_matrix[route[-1]][route[0]] += Q / route_length  # Closing the loop

    return best_route, best_length

# Run the ACO
best_route, best_length = ant_colony_optimization()
print(f"Best Route: {best_route}, Length: {best_length}")


Best Route: [1, 3, 4, 0, 2], Length: 184


This code implements an **Ant Colony Optimization (ACO)** algorithm to solve the **Traveling Salesman Problem (TSP)** with a small set of cities. Here's a breakdown of how it works, step-by-step:

---

## **Explanation of the Code**

### 1. **Constants and Initialization**

```python
NUM_CITIES = 5       # Number of cities in the TSP problem
NUM_ANTS = 10        # Number of ants to simulate
NUM_ITERATIONS = 100 # Number of iterations to run the algorithm
EVAPORATION_RATE = 0.5  # Rate at which pheromones evaporate
ALPHA = 1.0          # Influence of pheromone levels
BETA = 2.0           # Influence of distance (inverse)
Q = 100              # Constant for pheromone deposit strength

# Random distance matrix with values between 1 and 100
dist_matrix = np.random.randint(1, 100, size=(NUM_CITIES, NUM_CITIES))
np.fill_diagonal(dist_matrix, 0)  # Ensure the diagonal is zero (no distance to itself)

# Initialize pheromone matrix with ones
pheromone_matrix = np.ones((NUM_CITIES, NUM_CITIES))
```

- **`dist_matrix`**: Represents the distances between the cities. Random integers between 1 and 100 are assigned, with the diagonal set to 0.
- **`pheromone_matrix`**: Tracks the pheromone levels between cities, initially set to 1.

---

### 2. **Ant Colony Optimization Function**

#### **Main Algorithm**

```python
def ant_colony_optimization():
    global pheromone_matrix  # Use the global pheromone matrix
    best_route = None
    best_length = float('inf')
```

- **`best_route`** and **`best_length`**: Track the shortest route and its length found so far.

#### **Ant Simulation Loop**

```python
    for iteration in range(NUM_ITERATIONS):
        all_routes = []
        all_lengths = []

        for ant in range(NUM_ANTS):
            route = [random.randint(0, NUM_CITIES - 1)]
```

- For each iteration, each ant starts from a random city.

#### **Constructing the Route**

```python
            while len(route) < NUM_CITIES:
                current_city = route[-1]
                probabilities = []

                for next_city in range(NUM_CITIES):
                    if next_city not in route:
                        pheromone = pheromone_matrix[current_city][next_city] ** ALPHA
                        distance = dist_matrix[current_city][next_city]
                        if distance > 0:
                            distance = distance ** (-BETA)
                        else:
                            distance = 0
                        probabilities.append(pheromone * distance)
                    else:
                        probabilities.append(0)
```

- For each unvisited city:
  - Calculate the **pheromone influence** (`pheromone_matrix[current_city][next_city] ** ALPHA`).
  - Calculate the **distance influence** (`distance ** (-BETA)`).
  - Append the product of these values to the `probabilities` list.

#### **Selecting the Next City**

```python
                total_prob = sum(probabilities)
                if total_prob == 0:
                    break  # No valid move

                probabilities = [p / total_prob for p in probabilities]
                next_city = np.random.choice(range(NUM_CITIES), p=probabilities)
                route.append(next_city)
```

- Normalize the probabilities and randomly select the next city based on these probabilities.

#### **Calculate Route Length**

```python
            route_length = sum([dist_matrix[route[i]][route[i + 1]] for i in range(NUM_CITIES - 1)]) + \
                           dist_matrix[route[-1]][route[0]]  # Closing the loop
```

- The total length of the route is calculated by summing up the distances between consecutive cities and closing the loop back to the starting city.

#### **Track the Best Route**

```python
            all_routes.append(route)
            all_lengths.append(route_length)

            if route_length < best_length:
                best_length = route_length
                best_route = route
```

- Track the route and its length. Update the best route if a shorter one is found.

#### **Pheromone Update**

```python
        # Evaporate pheromones
        pheromone_matrix *= (1 - EVAPORATION_RATE)

        # Deposit new pheromones
        for route, route_length in zip(all_routes, all_lengths):
            for i in range(NUM_CITIES - 1):
                pheromone_matrix[route[i]][route[i + 1]] += Q / route_length
            pheromone_matrix[route[-1]][route[0]] += Q / route_length  # Closing the loop
```

- Pheromones evaporate by a fixed rate.
- Pheromones are deposited along each edge of the route, proportional to the inverse of the route length.

---

### 3. **Running the ACO**

```python
# Run the ACO
best_route, best_length = ant_colony_optimization()
print(f"Best Route: {best_route}, Length: {best_length}")
```

- Runs the ACO algorithm and prints the best route and its length.

---

## **Sample Output**

```plaintext
Best Route: [2, 4, 3, 1, 0], Length: 215
```

The output shows the best route found and the total distance of that route.

---

## **Considerations and Enhancements**

1. **Visualization**: Add visualization of the cities and routes to see the paths explored.
2. **Parameter Tuning**: Experiment with `NUM_ANTS`, `NUM_ITERATIONS`, `ALPHA`, `BETA`, and `EVAPORATION_RATE` to find optimal settings.
3. **Dynamic Distance Matrix**: Instead of a random matrix, use real-world coordinates and calculate distances.
4. **Handling Stagnation**: Introduce strategies to avoid getting stuck in local optima, such as pheromone resetting or adaptive parameters.